In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00a 0:00:01


In [3]:
%pwd

'/Users/ericdanforth/Downloads'

In [49]:
import PyPDF2
import csv
import pandas as pd

text_list = []
with open('2022_ca_designer_collection_1st_ptg_rev.pdf', 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    for page_number in range(50): #reader.numPages
        text = reader.pages[page_number].extract_text()
        text_list.append([text])

df = pd.DataFrame(text_list, columns=['Text'])
df.to_csv('ca_codes.csv', index=False)

In [50]:
import re
import pandas as pd

# Load the DataFrame from the CSV
# df = pd.read_csv('output.csv')

def process_text(row):
    text = row['Text']
    page_number = None

    if row.name % 2 == 0:  # Even page
        match = re.search(r'(.*?)(\n?)(2022 CALIFORNIA ADMINISTRATIVE CODE )(\d+-\d+)(.*)(Copyright © 2022 ICC)', text, re.DOTALL)
        if match:
            page_number = match.group(4)
            text = match.group(5)
    else:  # Odd page
        match = re.search(r'(.*?)(\n?)(\d+-\d+)( 2022 CALIFORNIA ADMINISTRATIVE CODE)(.*)(Copyright © 2022 ICC)', text, re.DOTALL)
        if match:
            text = match.group(5)
            page_number = match.group(3)

    # Return a new row
    return pd.Series([page_number, text])

# Apply the function to every row in the DataFrame
df[['Page', 'Text']] = df.apply(process_text, axis=1)

# Write the DataFrame back to the CSV
df.to_csv('ca_codes.csv', index=False)
df.head(20)

,Text,Page
0,"TO VIEW, CLICK ON A TITLE.\nHELPFUL LINKS\n202...",None
1,2022 CALIFORNIA \nADMINISTRATIVE CODE\nCALIF...,None
2,IMPORTANT NOTICE\nAct now to keep your code up...,None
3,2022 California Administrative Code\nCaliforn...,None
4,2022 CALIFORNIA ADMINISTRATIVE CODE iiiPREFACE...,None
5,iv 2022 CALIFORNIA ADMINISTRATIVE CODE\nCopyri...,None
6,2022 CALIFORNIA ADMINISTRATIVE CODE vCALIFORNI...,None
7,vi 2022 CALIFORNIA ADMINISTRATIVE CODEHOW TO D...,None
8,2022 CALIFORNIA ADMINISTRATIVE CODE viiTABLE O...,None
9,viii 2022 CALIFORNIA ADMINISTRATIVE CODETABLE ...,None


In [60]:
chap1 = []
for pagenum in range(12,23):
    content = df.iloc[pagenum][0]
    chap1.append(content)
concatenated_string = ''.join(chap1)

In [305]:
concatenated_string

'CHAPTER 1\nADMINISTRATIVE REGULATIONS OF THE \nCALIFORNIA BUILDING STANDARDS COMMISSION\nARTICLE 1\nGENERAL \n1-101. Abbreviations. The following abbreviations shall\napply to Title 24, California Code of Regulations. Abbrevia-\ntions may also be provided in  each of the other 12 parts of\nTitle 24. If an abbreviation in this section conflicts with an\nabbreviation within another part of Title 24, the abbreviation\nreference in the other part shall prevail within that part.\nNote: For information regarding the code provisions adopted\nfor a state agency and the appli cation of such code provisions,\nsee the agency administrative  chapters in the appropriate\nparts of Title 24,  California Code of Regulations. \nAGR Department of Food and Agriculture\nBSC Identifies code provisions by the Building Stan-\ndards Commission\nBSC-CG Identifies the Califo rnia Green Building Stan-\ndards Code (CALGreen) provisions by BSC \nBSCC Identifies code provisio ns by the Board of State\nand Communit

In [318]:
import re

def parse_text(text, level=0):
    patterns = [
        r"(CHAPTER \d+)(.*?)(ARTICLE \d.*?)(?=CHAPTER \d+|$)",
        r"(ARTICLE \d+)(.*?)(\d+-\d+\..*?)(?=ARTICLE \d+|$)",
        r"(\d+-\d+\.)(.*?\.)(.*?)(?=\d+-\d+\.|$)",
        r"\n(\([a-z]\)\s)(.*?)(.*?)(?=\n\([a-z]\)\s|$)",
        r"\n(\d+\.\s)(.*?)(.*?)(?=\n\d+\.\s|$)",  
        r"\n([A-Z]\s)(.*?\.)(.*?)(?=[A-Z]\w*|$)",
        r"\n\((\d+\s)\)(.*?\.)(.*?)(?=\(\d+\)|$)",
        r"\n\(([ivxlcdm]+\s)\)(.*?\.)(.*?)(?=\([ivxlcdm]+\)|$)" 
    ]

    # Base case: if level is equal to or exceeds the length of patterns, return text in a list
    if level >= len(patterns):
        return [[text.replace('-\n', '').replace('\n', ' ').strip(), '']]

    result = []
    match_list = re.findall(patterns[level], text, re.DOTALL)

    # If there's no match, go to the next level
    if len(match_list) == 0:
        return parse_text(text, level + 1)

    for match in match_list:
        pattern, title, following_text = match[0], match[1], match[2]
#         if level == 3:
#             print(f"pattern: {pattern}, title: {title}, following_text: {following_text[:300]}")
        # Apply the same parsing process for the rest of the text by increasing level
        remaining_parts = parse_text(following_text, level + 1)
        result.append([pattern.replace('-\n', '').replace('\n', ' ').strip(), title.replace('-\n', '').replace('\n', ' ').strip()])
        result.append(remaining_parts)
    return result



In [319]:
parsed_text = parse_text(concatenated_string)


In [320]:
import json

formatted_dict = json.dumps(parsed_text, indent=4)

print(formatted_dict)

[
    [
        "CHAPTER 1",
        "ADMINISTRATIVE REGULATIONS OF THE  CALIFORNIA BUILDING STANDARDS COMMISSION"
    ],
    [
        [
            "ARTICLE 1",
            "GENERAL"
        ],
        [
            [
                "1-101.",
                "Abbreviations."
            ],
            [
                [
                    "The following abbreviations shall apply to Title 24, California Code of Regulations. Abbreviations may also be provided in  each of the other 12 parts of Title 24. If an abbreviation in this section conflicts with an abbreviation within another part of Title 24, the abbreviation reference in the other part shall prevail within that part. Note: For information regarding the code provisions adopted for a state agency and the appli cation of such code provisions, see the agency administrative  chapters in the appropriate parts of Title 24,  California Code of Regulations.  AGR Department of Food and Agriculture BSC Identifies code provisions by the